In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]==0.8.0

In [ ]:
# !pip install snowflake-connector-python
!pip uninstall cloudpickle -y
!pip install cloudpickle==2.2.1

In [29]:
from snowflake.snowpark import Session

In [30]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWINSIGHT"

In [31]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=REGISTRY_DATABASE_NAME,
                         schema=REGISTRY_SCHEMA_NAME, 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [32]:
session = get_session()
# session.close()
session

In [28]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [ ]:
!pip list | grep -i sci

In [ ]:
session.sql("LIST STAGES")

In [ ]:
# model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [33]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [37]:
df = registry.show_models()
df
# type(df)

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-05-30 03:54:17.224000-07:00,SAMPLE_SKLEARN_MODEL,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
1,2024-05-31 02:58:30.562000-07:00,SAMPLE_SKLEARN_MODEL_2,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
2,2024-05-31 03:59:03.745000-07:00,SAMPLE_SKLEARN_MODEL_3,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
3,2024-06-02 22:54:16.932000-07:00,SAMPLE_SKLEARN_MODEL_4,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
4,2024-06-03 06:22:18.501000-07:00,SAMPLE_SKLEARN_MODEL_5,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"


# Get model list from snowpark registry

In [36]:
model_list = registry.models()
model_list

# Get model details from registry

In [ ]:
m =registry.get_model("sample_sklearn_model")
m

# Get model versions

In [ ]:
df_version = m.show_versions()
df_version

# Set default model version

In [ ]:
# default_version = m.default
m.default = "V1"

# Get default model version details from model-registry

In [ ]:
mv = m.default
mv

In [ ]:
x

In [ ]:
#Load default vesion as python object
# clf = mv.load()

# Run prediction on scikit-learn model

In [ ]:
import numpy as np
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = mv.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


In [ ]:
type(X_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

# Deploy model with custom class

In [ ]:
from snowflake.ml.model import custom_model

In [ ]:
class ExamplePipelineModel(custom_model.CustomModel):
#     def __init__(self, context: ModelContext) -> None:
#         super().__init__(context)
#         v = int(open(context.path('config')).read())
#         self.bias = json.loads(v)['bias']

    @custom_model.inference_api
    def run(self, data_list: list) -> pd.DataFrame:
#         features = self.context.model_ref('feature_preproc').transform(input)
#         model_output = self.context.model_ref('m2').predict(
#             self.context.model_ref('m1').predict(features)
#         )
#         return pd.DataFrame({
#             'output': model_output + self.bias})
        data_array = np.asarray(data_list)
        try:
            prediction = model.predict(data_array)
        except:
            prediction = model.predict(data_array.reshape(1, -1))
        return prediction.tolist()

In [ ]:
!pip freeze | grep cloud

# Train & register model

In [23]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [24]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

,short_name,nationality,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Argentina,94,94,565000,95500000,32,170,72,88,...,75,96,33,37,26,6,11,15,14,8
1,Cristiano Ronaldo,Portugal,93,93,405000,58500000,34,187,83,84,...,85,95,28,32,24,7,11,15,14,11
2,Neymar Jr,Brazil,92,92,290000,105500000,27,175,68,87,...,90,94,27,26,29,9,9,15,15,11
3,J. Oblak,Slovenia,91,93,125000,77500000,26,188,87,13,...,11,68,27,12,18,87,92,78,90,89
4,E. Hazard,Belgium,91,91,470000,90000000,28,175,74,81,...,88,91,34,27,22,11,12,6,8,8
5,K. De Bruyne,Belgium,91,91,370000,90000000,28,181,70,93,...,79,91,68,58,51,15,13,5,10,13
6,M. ter Stegen,Germany,90,93,250000,67500000,27,187,85,18,...,25,70,25,13,10,88,85,88,88,90
7,V. van Dijk,Netherlands,90,91,200000,78000000,27,193,92,53,...,62,89,91,92,85,13,10,13,11,11
8,L. Modric,Croatia,90,90,340000,45000000,33,172,66,86,...,82,92,68,76,71,13,9,7,14,9
9,M. Salah,Egypt,90,90,240000,80500000,27,175,71,79,...,77,91,38,43,41,14,14,9,11,14


In [ ]:
type(y_train)

In [25]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [26]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

GradientBoostingRegressor()

In [ ]:
gbm_default.head()

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_4")

# Generating model signature

In [ ]:
from snowflake.ml.model import model_signature

In [ ]:
sig = model_signature.infer_signature(
    X_train.head(),
    data['value_eur'],
    input_feature_names=['age','height_cm','weight_kg','skill_dribbling',
                         'attacking_crossing','attacking_finishing','movement_acceleration',
                         'movement_sprint_speed','power_shot_power','mentality_aggression',
                         'defending_marking','defending_standing_tackle','goalkeeping_diving',
                         'goalkeeping_handling','overall','potential'],
    output_feature_names=['value_eur'])

In [34]:
mv = registry.log_model(gbm_default,
                   model_name="sample_sklearn_model_5",
                   version_name="v1",
                   conda_dependencies=["scikit-learn==1.3.2", "scipy==1.13.1", "cloudpickle==2.2.1"],
                   comment="My awesome ML model",
                    python_version="3.9.19",
                   metrics={"score": 96},
                   sample_input_data=X_train
                       )

/opt/conda/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/tmp/pip_packages/snowflake/ml/model/model_signature.py:69: UserWarning: The sample input has 3750 rows, thus a truncation happened before inferring signature. This might cause inaccurate signature inference. If that happens, consider specifying signature manually.
  warnings.warn(


In [35]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [38]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_5")

# Create stage in snowflake-model-registry

In [ ]:
# query = """
#         create or replace stage sample_model_serving_udf_stg
#         directory = (enable = true)
#         copy_options = (on_error='skip_file')
#         """

# session.sql(query)



In [ ]:
# To see the stages
session.sql("show stages")

In [ ]:
from snowflake.snowpark.files import SnowflakeFile

In [ ]:
import cloudpickle
with SnowflakeFile.open("@STG_ML_MODELS/ml_model_2") as file:
    m= cloudpickle.load(file)
    print(type(m))

# Upload model file on ML_MODELS stage 

In [ ]:
#create the stage for storing the ML models
# session.sql("USE MODEL_REGISTRY_SNOWSIGHT;")
session.sql('CREATE OR REPLACE STAGE STG_ML_MODELS').show()

In [ ]:
session.file.put(
    X_train.to_csv(), "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

In [39]:
session.file.put(
    "/notebooks/notebooks/ml_model_5", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

[PutResult(source='ml_model_5', target='ml_model_5', source_size=181708, target_size=181712, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

# Read model file from stage

In [ ]:
session.clear_imports()
session.clear_packages()

#Register above uploded model as import of UDF
session.add_import("@STG_ML_MODELS/ml_model_2")

#map packege dependancies
session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")

In [ ]:
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
import snowflake.snowpark.functions as F
from typing import  Optional



def read_file(filename):
    import cloudpickle
    import sys
    import os
    
    #where all imports located at
    import_dir = sys._xoptions.get("snowflake_import_directory")

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = cloudpickle.load(file)
            return m

#register UDF
@F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
def read_model_from_file(file_name:Optional[str]) -> str:
    import numpy as np
    
    # later we will input train data as JSON object
    # hance, we have to convert JSON object as pandas DF
    data_array = np.asarray(data_list)
    pipeline = read_file(file_name)
    print("type of pickle object =", type(pipeline))
#     dict_mod["model_obj"] = pipeline
    return "pipeline"
#     return pipeline.predict(data_array).to_list()




# @F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
# def read_model_from_file(file_name:Optional[str]) -> str:
#     import numpy as np
    
#     # later we will input train data as JSON object
#     # hance, we have to convert JSON object as pandas DF
#     data_array = np.asarray(data_list)
#     pipeline = read_file(file_name)
#     print("type of pickle object =", type(pipeline))
# #     dict_mod["model_obj"] = pipeline
#     return pipeline
# #     return pipeline.predict(data_array).to_list()

In [ ]:
session.sql("DESC FUNCTION READ_MODEL_FROM_FILE("ml_model")").show()

In [ ]:
model_udf = read_model_from_file("ml_model_2")
# model_udf = read_file("ml_model_2")

In [ ]:
model_udf

In [ ]:
registry.

In [ ]:
import joblib

In [ ]:
# model_obj = joblib.load(read_model_from_file("ml_model_2"))
import os
model_file = os.path.join("/notebooks/notebooks", "ml_model_3")
model_obj = joblib.load(model_file)

In [ ]:
X_train.to_csv("/data/X_train.csv")

In [ ]:
import numpy as np

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = model_obj.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
from snowflake.snowpark import DataFrame

In [40]:
import pandas as pd

In [41]:
from snowflake.snowpark.types import Variant
def read_model_from_stage(session: Session, df_dict: dict) -> Variant:

    # print("file_name =", file_name)
    # session.add_import("@STG_ML_MODELS/"+file_name)
    # map packege dependancies
    # session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")



    # from snowflake.snowpark.types import PandasSeries, PandasDataFrame
    # import snowflake.snowpark.functions as F
    # from typing import Optional
    from snowflake.snowpark.files import SnowflakeFile
    from snowflake.ml.registry.registry import Registry

    import joblib
    df = pd.DataFrame(df_dict)
    print("df ===", df)

    model_file = '@STG_ML_MODELS/ml_model_5'
    # Specify 'mode = rb' to open the file in binary mode.
    with SnowflakeFile.open(model_file, 'rb', require_scoped_url=False) as f:
        model_obj = joblib.load(f)
        # return str(model_obj)
        # return str(model_obj)

        reg = Registry(session=session)

        mv = reg.log_model(model=model_obj,
                           model_name="sample_model",
                           comment="test",
                           version_name="run1",
                           python_version="3.9.19",
                           conda_dependencies=["scikit-learn==1.3.2"],
                           metrics={"model_metrics": {"score": 96}, "project_id": "0001", "type": "Model"},
                           sample_input_data=df
                           )

        print("model got registered successfully")
        return df_dict

In [42]:
session.sproc.register(func=read_model_from_stage,
                           name="read_model_from_stage",
                           packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn", "pandas", "cloudpickle==2.2.1", "joblib"],
                           isPermanant=True,
                           stage_location="@STG_ML_MODELS",
                           replace=True,
                       )

In [49]:
dict_df = X_trainhead().to_dict()

NameError: name 'head' is not defined

In [48]:
dict_df

{'age': {620: 25,
  2193: 26,
  1170: 32,
  1863: 34,
  4724: 24,
  2098: 22,
  769: 25,
  1200: 28,
  3539: 35,
  3035: 29,
  1349: 28,
  2801: 27,
  1464: 30,
  4859: 27,
  4924: 27,
  1938: 28,
  2997: 28,
  3376: 26,
  1047: 24,
  88: 28,
  3386: 25,
  2611: 21,
  3993: 23,
  1497: 27,
  4951: 27,
  2647: 25,
  49: 30,
  2704: 25,
  1361: 27,
  3814: 27,
  2187: 25,
  3045: 30,
  546: 27,
  2831: 27,
  4396: 30,
  1162: 31,
  1937: 29,
  1499: 28,
  2912: 30,
  3348: 25,
  1323: 25,
  1672: 24,
  4082: 26,
  400: 30,
  4162: 27,
  3824: 35,
  2261: 27,
  821: 28,
  780: 22,
  4306: 32,
  315: 28,
  4273: 32,
  1264: 22,
  2778: 25,
  286: 36,
  2427: 32,
  731: 28,
  1918: 29,
  3615: 30,
  2667: 23,
  2765: 26,
  2945: 30,
  1045: 24,
  160: 20,
  2975: 35,
  1473: 31,
  3088: 31,
  539: 29,
  294: 33,
  3959: 23,
  1768: 29,
  1457: 29,
  116: 25,
  614: 25,
  2049: 20,
  3657: 28,
  3387: 26,
  2052: 19,
  3608: 28,
  3398: 27,
  4414: 29,
  1214: 18,
  3701: 31,
  365: 27,
  12

In [44]:
X_train.columns

Index(['age', 'height_cm', 'weight_kg', 'skill_dribbling',
       'attacking_crossing', 'attacking_finishing', 'movement_acceleration',
       'movement_sprint_speed', 'power_shot_power', 'mentality_aggression',
       'defending_marking', 'defending_standing_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'overall', 'potential'],
      dtype='object')

In [45]:
df = pd.DataFrame(dict_df)

In [46]:
df.columns

Index(['age', 'height_cm', 'weight_kg', 'skill_dribbling',
       'attacking_crossing', 'attacking_finishing', 'movement_acceleration',
       'movement_sprint_speed', 'power_shot_power', 'mentality_aggression',
       'defending_marking', 'defending_standing_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'overall', 'potential'],
      dtype='object')

In [47]:
model_obj = session.call("read_model_from_stage", dict_df)

SnowparkSQLException: (1304): 01b4c345-0000-668c-0000-576d06ccab2a: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 51, in compute
  File "<ipython-input-41-6d7549348a0d>", line 30, in read_model_from_stage
  File "/usr/lib/python_udf/c9abc2f36199647a270d8ce824ea98bb92e1209b64d0ac5efb8c6c1d19a3e362/lib/python3.9/site-packages/snowflake/ml/_internal/telemetry.py", line 352, in wrap
    return update_stmt_params_if_snowpark_df(func(*args, **kwargs), statement_params)
  File "/usr/lib/python_udf/c9abc2f36199647a270d8ce824ea98bb92e1209b64d0ac5efb8c6c1d19a3e362/lib/python3.9/site-packages/snowflake/ml/registry/registry.py", line 156, in log_model
    return self._model_manager.log_model(
  File "/usr/lib/python_udf/c9abc2f36199647a270d8ce824ea98bb92e1209b64d0ac5efb8c6c1d19a3e362/lib/python3.9/site-packages/snowflake/ml/registry/_manager/model_manager.py", line 71, in log_model
    mc.save(
  File "/usr/lib/python_udf/c9abc2f36199647a2

In [2]:
!pip freeze | grep -i categ

In [ ]:
!pip install category_encoders

In [3]:
import pandas as pd
import numpy as np
# import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
# from refractml import *
# from refractml.constants import MLModelFlavours

In [4]:
cars = pd.read_csv("/data/car_new.csv")
cars

,buying_num,maintenance_num,doors_num,persons_num,lugboot_num,safety_num,class_num
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [5]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1416, 6) (311, 6) (1416,) (311,)


In [6]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier()

In [7]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [8]:
model.predict(X_train.head(1))

array([2])

In [9]:
from snowflake.ml.model import model_signature

In [21]:
y_train.head().tolist()

[2, 2, 0, 2, 2]

In [22]:
sig = model_signature.infer_signature(
    X_train.head(),
    y_train.head().tolist(),
    input_feature_names=X_train.columns.tolist(),
    output_feature_names=['class_num'])

# 